##### import

In [1]:
import os
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from stock_centre import *
import scipy.stats as si
import plotly.express as px
from opt_assistant import *
hq = Stock_Data_Centre()
pro = TuShare().pro
opt_info = Opt_Assistant()

##### set global param

In [2]:
underlying_symbol = '510050'
is_index = False
contract_types = ['CO','PO', 'CO']
trade_num = [-1, 2, -1]
gears = [2, 2, 2] #虚值和实值的档位，虚（实）值为正（负），如虚（实）一时， n=1（-1）
bail_adjust = 0.12
min_multiple = 0.5
init_cash = 1000000
max_cash = 3000000
trade_logic = 1
date_select = 4
start_date = '2021-07-01'
end_date = '2023-06-30'

#### 一、回测前函数定义

##### 标准化函数

In [3]:
def param_standard(underlying_symbol):

    global start_date
    global end_date

    #  标的范围
    index_list = ['000300', '000852', '000016']
    etf_list = ['510050', '510300', '510500', '588000', '588080', '159915', '159901', '159919', '159922']

    #  标的不同系统中的代码和历史价格和
    if underlying_symbol in index_list:  # 指数的交易所为上交所
        underlying_symbol_for_stock = underlying_symbol + '.SH'
        underlying_symbol_for_opt = underlying_symbol + '.XSHG'
        underlying_prc_his = hq.get_hq(code=underlying_symbol_for_stock, start_date=start_date, end_date=end_date,
                                       index_data=True)
        handling_fee = 15
        commission = 15 * 3
    elif underlying_symbol in etf_list:  # etf的交易所按开头分类
        if underlying_symbol.startswith('5'):
            underlying_symbol_for_stock = underlying_symbol + '.SH'
            underlying_symbol_for_opt = underlying_symbol + '.XSHG'
        elif underlying_symbol.startswith('1'):
            underlying_symbol_for_stock = underlying_symbol + '.SZ'
            underlying_symbol_for_opt = underlying_symbol + '.XSHE'
        underlying_prc_his_Tushare = pro.fund_daily(ts_code=underlying_symbol_for_stock,
                                                    start_date=pd.to_datetime(start_date, format='%Y-%m-%d').strftime(
                                                        '%Y%m%d'),
                                                    end_date=pd.to_datetime(end_date, format='%Y-%m-%d').strftime(
                                                        '%Y%m%d'))
        underlying_prc_his = underlying_prc_his_Tushare.set_index(
            pd.to_datetime(underlying_prc_his_Tushare['trade_date'], format='%Y%m%d').dt.strftime('%Y-%m-%d'))
        underlying_prc_his.sort_index(inplace=True)
        handling_fee = 1.3 + 0.3
        commission = 5

    per_fee = handling_fee + commission

    return {'underlying_symbol_for_stock': underlying_symbol_for_stock, 
            'underlying_symbol_for_opt': underlying_symbol_for_opt, 'per_fee': per_fee, 
            'underlying_prc_his': underlying_prc_his}

a = param_standard(underlying_symbol)['underlying_prc_his']
a[a.index == '2021-07-05']
# a['trade_date'] = pd.to_datetime(a['trade_date'])
# a[a['trade_date'] == '2021-07-03'].empty
# a.sort_values(['open'], ascending=False).iloc[1,:]['open']
# a[a.index == '2021-07-05']

,ts_code,trade_date,pre_close,open,high,low,close,change,pct_chg,vol,amount
trade_date,,,,,,,,,,,
2021-07-05,510050.SH,20210705,3.423,3.419,3.435,3.393,3.42,-0.003,-0.0876,4262801.81,1455122.932


##### 交易日函数

In [4]:
def get_trade_dates(underlying_symbol, start_date, end_date):
    '''
    获取某标的在指定日期间的交易日序列
    :param underlying_symbol: 标的代码，仅数字，str格式
    :param start_date: 开始日期
    :param end_date: 截止日期
    :return: # list格式，list中形式为：str '2023-06-16'
    '''
    
    ts_prc_his = param_standard(underlying_symbol)['underlying_prc_his']
    # print(pd.to_datetime(start_date).strftime('%Y-%m-%d'), pd.to_datetime(end_date).strftime('%Y-%m-%d'))
    
    if pd.to_datetime(start_date).strftime('%Y-%m-%d') <= pd.to_datetime(end_date).strftime('%Y-%m-%d'):
        ts_prc_his = ts_prc_his[(ts_prc_his.index >= pd.to_datetime(start_date).strftime('%Y-%m-%d'))
                            & (ts_prc_his.index <= pd.to_datetime(end_date).strftime('%Y-%m-%d'))]
        dates_info = ts_prc_his.index.tolist()
        return dates_info
    else:
        print('起始日期晚于结束日期')
        return []

# print(get_trade_dates(underlying_symbol, start_date, end_date))

def get_trade_date_gap(underlying_symbol, start_date, end_date):
    '''
    获取某标的在指定日期间间隔的交易日天数
    :param underlying_symbol: 标的代码，仅数字，str格式
    :param start_date: 开始日期
    :param end_date: 截止日期
    :return: int格式
    '''
    dates_gap = len(get_trade_dates(underlying_symbol, start_date, end_date))
    return dates_gap

# print(get_trade_date_gap(underlying_symbol, '2023-01-12', '2023-01-30'))

def get_trade_date(underlying_symbol, spec_date):
    '''
    获取某标的距离指定日最近的交易日
    :param underlying_symbol: 指定标的
    :param spec_date: 指定日期，形式'%Y-%m-%d'
    :return:
    '''
    trade_date = get_trade_date(underlying_symbol, spec_date, spec_date)
    while len(trade_date) == 0: # 说明该日非交易日
        spec_date = pd.to_datetime(spec_date)
        spec_date  += relativedelta(days=1)
        trade_date = get_trade_date(underlying_symbol, spec_date, spec_date)

    return spec_date

def get_selected_date_of_month(start_date, end_date, date_select):
    '''
    获得时间区间内每月n号
    '''
    start_date, end_date = pd.to_datetime(start_date), pd.to_datetime(end_date)
    date_list = []
    if start_date.day > date_select:  # 如果开始日期已经过了，调整为下月
        current_date = datetime.datetime(start_date.year, start_date.month, date_select) + relativedelta(months=1)
    else:
        current_date = datetime.datetime(start_date.year, start_date.month, date_select)  # 将开始日期调整为该月的10号
    while current_date <= end_date:  # 循环遍历每个月，直到当前日期大于等于结束日期
        date_list.append(current_date)
        current_date += relativedelta(months=1)
    return date_list

def get_last_of_month(start_date, end_date):
    '''
    获得时间区间内每月最后一天
    '''
    start_date, end_date = pd.to_datetime(start_date), pd.to_datetime(end_date)
    date_list = []
    current_date = datetime.datetime(start_date.year, start_date.month, 1) + relativedelta(months=1) - datetime.timedelta(days=1)# 将开始日期调整为该月的最后一天    
    while current_date <= end_date:
        date_list.append(current_date)
        current_date += datetime.timedelta(days=1)  # 再调整为下一个月的最后一天，必须分开进行，否则死循环
        current_date += relativedelta(months=1)
        current_date -= datetime.timedelta(days=1)
    return date_list

def get_adj_date(date_list):
    '''
    将列表中的非交易日顺延
    '''
    adj_date = []
    trade_date_list = hq.get_trade_date(start_date=date_list[0], end_date=(date_list[-1] + relativedelta(months=1)), only_list=True)  # 获取区间内交易日，延长一月防止最后一个日期是非交易日
    for date in date_list:
        if date in trade_date_list:  # 如果列表内日期是交易日，不调整
            adj_date.append(date)
        else:  # 将trade_date_list里所有的日期与当前日期做差，找最小的正值，加在当前日期上
            for trade_day in trade_date_list:
                delta = trade_day - date
                if delta > datetime.timedelta(0):
                    date = date + delta
                    break
            adj_date.append(date)
    return adj_date

def get_ym(date, relative_month):
    date = pd.to_datetime(date)
    return (date + relativedelta(months=relative_month)).strftime("%Y-%m")

##### 保证金计算函数

In [5]:
def calculate_init_margin(code, date):
    '''
    计算指定合约在某日的开仓保证金水平
    :param code: 拟交易的期权合约代码，形式‘10004405.XSHG'
    :param date: 指定日期, str '2023-06-16'
    :param bail_adjust: 保证金调整系数
    :param min_multiple: 最低保障倍数
    :return:
    '''

    global hq
    global opt_info
    global underlying_symbol
    global bail_adjust
    global min_multiple
    opt_quotation_info = pd.read_feather(f"{hq.data_path}/quotation/opt/1d/{pd.to_datetime(date).strftime('%Y-%m-%d')}.fea")
    temp_opt_prc_info = opt_quotation_info[opt_quotation_info['code']==code]
    temp_opt_info = opt_info.get_contract_info(code)
    ts_prc_info = param_standard(underlying_symbol)['underlying_prc_his']
    temp_ts_prc_info = ts_prc_info[ts_prc_info.index == pd.to_datetime(date).strftime('%Y-%m-%d')]

    contract_type = temp_opt_info['contract_type'].tolist()[0]
    contract_type = (contract_type=='CO') - (contract_type=='PO') # CO=1,PO=-1
    contract_unit = temp_opt_info['contract_unit'].tolist()[0]
    exercise_price = temp_opt_info['exercise_price'].tolist()[0]
    expire_date = temp_opt_info['expire_date'].tolist()[0]

    opt_pre_settle = temp_opt_prc_info['pre_settle'].tolist()[0]
    ts_pre_close = temp_ts_prc_info['pre_close'].tolist()[0]

    # 期权虚值额 = max(行权价-标的前收盘价, 0) * opt_type
    virtual_pre_value = np.max(exercise_price - ts_pre_close, 0) * contract_type

    # 每手看涨期权保证金=合约乘数*(合约前结算价+max(标的前收盘价×合约保证金调整系数-前虚值额， 最低保障系数×标的前收盘价×合约保证金调整系数))
    # 每手看跌期权保证金=合约乘数*(合约前结算价+max(标的前收盘价×合约保证金调整系数-前虚值额，最低保障系数×合约行权价格×合约保证金调整系数）
    init_margin = contract_unit * (opt_pre_settle + bail_adjust * 
                                   max(ts_pre_close - virtual_pre_value,
                                       min_multiple * (ts_pre_close*(contract_type==1)
                                                       +exercise_price*(contract_type==-1))
                                       )
                                   )
    return init_margin

# print(calculate_init_margin('10004405.XSHG', '2023-03-21'))

def calculate_maintainance_margin(code, date):
    '''
    计算指定合约在某日的开仓保证金水平
    :param code: 拟交易的期权合约代码，形式‘10004405.XSHG'
    :param date: 指定日期, str '2023-06-16'
    :param bail_adjust: 保证金调整系数
    :param min_multiple: 最低保障倍数
    :return:
    '''

    global hq
    global opt_info
    global underlying_symbol
    global bail_adjust
    global min_multiple
    opt_quotation_info = pd.read_feather(f"{hq.data_path}/quotation/opt/1d/{pd.to_datetime(date).strftime('%Y-%m-%d')}.fea")
    temp_opt_prc_info = opt_quotation_info[opt_quotation_info['code']==code]
    temp_opt_info = opt_info.get_contract_info(code)
    ts_prc_info = param_standard(underlying_symbol)['underlying_prc_his']
    temp_ts_prc_info = ts_prc_info[ts_prc_info.index == pd.to_datetime(date).strftime('%Y-%m-%d')]

    contract_type = temp_opt_info['contract_type'].tolist()[0]
    contract_type = (contract_type=='CO') - (contract_type=='PO') # CO=1,PO=-1
    contract_unit = temp_opt_info['contract_unit'].tolist()[0]
    exercise_price = temp_opt_info['exercise_price'].tolist()[0]
    expire_date = temp_opt_info['expire_date'].tolist()[0]

    opt_settle = temp_opt_prc_info['settle_price'].tolist()[0]
    ts_close = temp_ts_prc_info['close'].tolist()[0]

    # 期权虚值额 = max(行权价-标的收盘价, 0) * opt_type
    virtual_pre_value = np.max(exercise_price - ts_close, 0) * contract_type

    # 每手看涨期权保证金=合约乘数*(合约结算价+max(标的收盘价×合约保证金调整系数-虚值额， 最低保障系数×标的收盘价×合约保证金调整系数))
    # 每手看跌期权保证金=合约乘数*(合约结算价+max(标的收盘价×合约保证金调整系数-虚值额，最低保障系数×合约行权价格×合约保证金调整系数))
    maintainance_margin = contract_unit * (opt_settle + bail_adjust *
                                   max(ts_close - virtual_pre_value,
                                       min_multiple * (ts_close*(contract_type==1)
                                                       +exercise_price*(contract_type==-1))
                                       )
                                   )
    return maintainance_margin

# print(calculate_maintainance_margin('10004405.XSHG', '2023-03-21'))

##### 希腊字母计算函数

In [6]:
def calculate_greeks(code, date, interest_rate_type='1y', his_vol_len=20):
    '''
    计算一张期权的希腊字母（已乘乘数）
    :param code: 期权代码
    :param date: 需要求希腊值的日期
    :param interest_rate_type: Shibor的类型，范围：1w, 2w, 1m, 3m, 6m, 9m, 1y
    :param his_vol_len: int, 历史波动率的时间跨度（天）
    :return: 理论价格，delta，gamma，vega，theta，rho, 杠杆率
    '''

    global underlying_symbol

    period_end_date = pd.to_datetime(date, format='%Y-%m-%d')

    period_start_date = period_end_date.replace(year=period_end_date.year - 1)

    contract_info = opt_info.get_contract_info(code=code)
    
    daily_info = opt_info.get_daily_info(code=code, date=date)

    underlying_symbol_for_stock = param_standard(underlying_symbol=underlying_symbol)['underlying_symbol_for_stock']
    #  标的历史价格
    if contract_info['underlying_type'].iloc[0] == 'INDEX':
        underlying_prc_his = hq.get_hq(code=underlying_symbol_for_stock, start_date=period_start_date,
                                       end_date=period_end_date,
                                       index_data=True)
    elif contract_info['underlying_type'].iloc[0] == 'ETF':
        underlying_prc_his_Tushare = pro.fund_daily(ts_code=underlying_symbol_for_stock,
                                                    start_date=period_start_date.strftime('%Y%m%d'),
                                                    end_date=period_end_date.strftime('%Y%m%d'))
        underlying_prc_his = underlying_prc_his_Tushare.set_index(
            pd.to_datetime(underlying_prc_his_Tushare['trade_date'], format='%Y%m%d').dt.strftime('%Y-%m-%d'))
        underlying_prc_his.sort_index(ascending=False, inplace=True)
    
    contract_unit = contract_info['contract_unit'].iloc[0]
    
    s = underlying_prc_his['close'].iloc[0]

    k = contract_info['exercise_price'].iloc[0]
    
    r = pro.shibor(start_date=period_end_date.strftime('%Y%m%d'), end_date=period_end_date.strftime('%Y%m%d'))[
            interest_rate_type].iloc[0] / 100

    exercise_date = contract_info['exercise_date'].iloc[0]

    t = len(hq.get_trade_date(start_date=period_end_date, end_date=exercise_date, only_list=True)) / 365

    sigma = underlying_prc_his.head(his_vol_len)['close'].pct_change().std() * np.sqrt(252)

    if contract_info['contract_type'].iloc[0] == 'CO':
        n = 1
    elif contract_info['contract_type'].iloc[0] == 'PO':
        n = -1

    d1 = (np.log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)

    theo_price = round(contract_unit * n * (s * si.norm.cdf(n * d1) - k * np.exp(-r * t) * si.norm.cdf(n * d2)), 2)

    delta = round(contract_unit * n * si.norm.cdf(n * d1), 2)

    gamma = round(contract_unit * si.norm.pdf(d1) / (s * sigma * np.sqrt(t)), 2)

    vega = round(contract_unit * (s * si.norm.pdf(d1) * np.sqrt(t)) / 100, 2)

    theta = round(contract_unit * (-1 * (s * si.norm.pdf(d1) * sigma) / (2 * np.sqrt(t)) - n * r * k * np.exp(-r * t) * si.norm.cdf(n * d2)) / 365, 2)

    rho = round(contract_unit * n * (t * k * np.exp(-r * t) * si.norm.cdf(d2) * 0.01), 2)

    leverage = round(s * delta / daily_info['settle_price'].iloc[0], 2)

    return {'theo_price': theo_price, 'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta, 'rho': rho,
            'leverage': leverage}


# print(calculate_greeks('10004405.XSHG', '2023-03-21', interest_rate_type='1y', his_vol_len=20))

##### 逻辑1选择交易期权函数

In [7]:
def find_target_opt(full_opt_info, underlying_pre_close, gear):
    available_price_gears = full_opt_info['exercise_price'].tolist()
    low = 0
    high = len(available_price_gears) - 1
    while low <= high:
        mid = (low + high) // 2
        mid_value = available_price_gears[mid]
        if mid_value < underlying_pre_close:
            low = mid + 1
        else:
            high = mid - 1
    if gear <0:
        target_gear = high+gear+1    
    elif gear >0:
        target_gear = low+gear-1
    return full_opt_info.iloc[target_gear,:]


def get_trade_opt_for_logic1(date, contract_type, gear):
    '''
    选择下月到期期权
    :param date: 指定日期, str '2023-06-16'
    
    :return:
    '''

    global underlying_symbol
    global opt_info

    underlying_symbol_for_opt = param_standard(underlying_symbol)['underlying_symbol_for_opt']
    underlying_prc_his = param_standard(underlying_symbol)['underlying_prc_his']
    temp_prc_info = underlying_prc_his[underlying_prc_his.index == pd.to_datetime(date).strftime('%Y-%m-%d')]
    
    date = pd.to_datetime(date)
    exercise_ym = get_ym(date, relative_month=1)

    underlying_pre_close = temp_prc_info['pre_close'].tolist()[0]
    
    full_opt_info = opt_info.get_gear_info(underlying_symbol=underlying_symbol_for_opt, contract_type=contract_type, exercise_ym=exercise_ym, ascending=True, date=date.strftime('%Y-%m-%d'))
    target_opt_code = find_target_opt(full_opt_info, underlying_pre_close, gear).loc['code']

    return target_opt_code

##### 逻辑2选择交易期权函数

In [8]:
def get_trade_opt_for_logic2(date, contract_type, gear):
    '''
    选择理想的期权
    :param date: 指定日期, str '2023-06-16'
    :return:
    '''

    global underlying_symbol
    global date_select
    global opt_info

    underlying_symbol_for_opt = param_standard(underlying_symbol)['underlying_symbol_for_opt']
    ts_prc_info = param_standard(underlying_symbol)['underlying_prc_his']
    temp_ts_prc_info = ts_prc_info[ts_prc_info.index == pd.to_datetime(date).strftime('%Y-%m-%d')]
    
    date = pd.to_datetime(date)
    def get_month_opt(date):
        y_m = date.strftime("%Y-%m") # 当月合约
        flag = False

        while not flag:
            opts = opt_info.get_gear_info(underlying_symbol_for_opt, contract_type,
                                         exercise_ym=y_m, ascending=True, date=date)
            if opts.empty == False:
                expire_date = pd.to_datetime(opts['expire_date'].iloc[0])
                duration = get_trade_date_gap(underlying_symbol, date.strftime("%Y-%m-%d"), expire_date.strftime("%Y-%m-%d"))
                if duration > date_select:
                    flag = True
                else:
                    y_m = (date + relativedelta(months=1)).strftime("%Y-%m")

        return opts

    month_opts = get_month_opt(date)

    def get_exercise_opt(month_opts):
        ts_pre_close = temp_ts_prc_info['pre_close'].tolist()[0]
        if gear * ((contract_type=='CO') - (contract_type=='PO')) > 0: # 看涨期权的虚值、看跌期权的实值往上取
            exercise_opt = month_opts[month_opts['exercise_price'] > ts_pre_close].iloc[abs(gear)-1]
        elif gear * ((contract_type=='CO') - (contract_type=='PO')) < 0: # 看涨期权的虚值、看跌期权的实值往下取
            exercise_opt = month_opts[month_opts['exercise_price'] < ts_pre_close].iloc[-abs(gear)]

        exercise_opt_code = exercise_opt['code']

        return exercise_opt_code

    exercise_opt_code = get_exercise_opt(month_opts)
    return exercise_opt_code

# get_trade_opt_for_logic2('2023-01-12', 'CO', 2)

#### 二、盘前函数定义

In [9]:
def before_trade(date):
    '''
    盘前函数，给出交易信号
    :param date: 指定日期，str %Y-%m-%d
    :return:
    '''
    global trade_logic
    global contract_types
    global gears
    global end_date
    global date_select
    global underlying_symbol
    
    if os.path.exists('position.csv') == False: # 尚未开始回测, 开仓
        if trade_logic ==1:
            trade_opt_code = []
            if pd.to_datetime(date) in get_adj_date(get_selected_date_of_month(start_date, end_date, date_select)):# 确认是否为开仓日
                for i, contract_type in enumerate(contract_types):
                    gear = gears[i]
                    trade_opt_code.append(get_trade_opt_for_logic1(date, contract_type, gear))
                return {'is_open':True, 'is_close':False, 'trade_opt_code':trade_opt_code, 'close_opt_code':[]}
            else:
                return {'is_open': False, 'is_close': False,
                        'trade_opt_code': [], 'close_opt_code': []}
        if trade_logic == 2:
            trade_opt_code = []
            for i, contract_type in enumerate(contract_types):
                gear = gears[i]
                trade_opt_code.append(get_trade_opt_for_logic2(date, contract_type, gear))
            return {'is_open':True, 'is_close':False,
                    'trade_opt_code':trade_opt_code, 'close_opt_code':[]}

    else: # 读取前一日结果并给出交易信号
        backtest_remain_term = get_trade_date_gap(underlying_symbol, date, end_date)
        position = pd.read_csv('position.csv')
        last_date = pd.to_datetime(position.sort_values(['datetime'], ascending=True).iloc[-1,:]['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        position["datetime"] = pd.to_datetime(position["datetime"]).map(lambda x: datetime.datetime.strftime(x, '%Y-%m-%d %H:%M:%S'))
        latest_position = position[ position["datetime"] == last_date]
        
        if trade_logic == 1:
            if pd.to_datetime(date) in get_adj_date(get_selected_date_of_month(start_date, end_date, date_select)):
                need_close_opt = latest_position[latest_position['res_num'] > 0] # 取出需要展期的合约
            else:
                need_hold_opt = latest_position[latest_position['res_num'] > 0] # 取出可以继续持有的合约
            if need_close_opt.empty == False: # 有需要展期(含平仓/开仓）的合约
                close_opt_code = need_close_opt['code'].tolist()
                trade_opt_code = []
                for i, contract_type in enumerate(contract_types):
                    gear = gears[i]
                    trade_opt_code.append(get_trade_opt_for_logic1(date,contract_type, gear))
                return {'is_open':True, 'is_close':True,
                    'trade_opt_code':trade_opt_code, 'close_opt_code':close_opt_code}
            else:
                return {'is_open': False, 'is_close': False,
                        'trade_opt_code': [], 'close_opt_code': []}

        if trade_logic == 2:
            need_close_opt = latest_position[(latest_position['expire_days'] == date_select) & (latest_position['res_num'] > 0)] # 取出需要展期的合约
            need_hold_opt = latest_position[latest_position['expire_days'] > date_select & (latest_position['res_num'] > 0)] # 取出可以继续持有的合约
            if (need_close_opt.empty == False) and (backtest_remain_term > date_select): # 有需要展期(含平仓/开仓）的合约
                close_opt_code = need_close_opt['code'].tolist()
                trade_opt_code = []
                for i, contract_type in enumerate(contract_types):
                    gear = gears[i]
                    trade_opt_code.append(get_trade_opt_for_logic2(date,contract_type, gear))
                return {'is_open':True, 'is_close':True,
                    'trade_opt_code':trade_opt_code, 'close_opt_code':close_opt_code}
            elif ((need_close_opt.empty == False) and (backtest_remain_term <= date_select)) or ((need_hold_opt.empty == False) and (backtest_remain_term <= 1)): 
                # 平仓且不再展期（无新开仓）
                if need_close_opt.empty == False:
                    close_opt_code = need_close_opt['code'].tolist()
                elif need_hold_opt.empty == False:
                    close_opt_code = need_hold_opt['code'].tolist()
                return {'is_open': False, 'is_close': True,
                        'trade_opt_code': [], 'close_opt_code': close_opt_code}
            else:
                return {'is_open': False, 'is_close': False,
                        'trade_opt_code': [], 'close_opt_code': []}
        
signal = before_trade('2021-07-05')
print(signal)

{'is_open': True, 'is_close': False, 'trade_opt_code': ['10003466.XSHG', '10003475.XSHG', '10003466.XSHG'], 'close_opt_code': []}


#### 三、盘中函数定义(平仓部分)

##### 平仓函数

In [26]:
# dict1 = {'is_open': False, 'is_close': True, 'trade_opt_code': [], 'close_opt_code': ['10003420.XSHG', '10003426.XSHG']}
# date = datetime.datetime(2021, 7, 1, 12, 2, 3).strftime('%Y-%m-%d %H:%M:%S')
# # print(date)
# position = pd.DataFrame({'datetime': [date, date, date], 
#                          'code': ['10003420.XSHG', '10003426.XSHG', '10003421.XSHG'], 
#                          'underlying_symbol':['510050', '510050', '510050'],
#                          'exercise_price':[3.7,3.4,3.7], 
#                          'expire_date': ['2021-07-28','2021-07-28','2021-07-28'], 
#                          'expire_days': [29,29,29], 
#                          'res_num': [3,3,3], 
#                          'underlying_close': [3.5,3.5,3.5], 
#                          'opt_settle':[3,3,3], 
#                          'hold_pl':[0,0,0],
#                          'acc_hold_pl':[0,0,0],
#                          'trade_pl':[0,0,0],
#                          'init_margin':[0,0,0],
#                          'maintainance_margin':[1,2,3],
#                          'theo_price':[1,1,1], 
#                          'delta':[1,1,1], 
#                          'gamma':[1,1,1], 
#                          'vega':[1,1,1], 
#                          'theta':[1,1,1], 
#                          'rho':[1,1,1], 
#                          'leverage':[1,2,1]
#                         })
# position.to_csv('position.csv', index = False)

# account = pd.DataFrame({ 'date': ['2021-06-30', '2021-07-01', '2021-07-02'], 
#                          'init_cash': [100,100,100], 
#                          'add_cash':[0,100,0],
#                          'total_cash':[100,200,200], 
#                          'hold_pl': [0,0,0], 
#                          'acc_hold_pl': [29,29,29], 
#                          'margin_occup': [100000,100000,100000], 
#                          'cash': [100000,100000,100000], 
#                          'margin_account':[200000,200000,200000], 
#                          'bail_risk_rate':[0,0,0],
#                          'delta':[0,0,0],
#                          'gamma':[0,0,0],
#                          'vega':[0,0,0],
#                          'theta':[1,2,3],
#                          'rho':[1,1,1], 
#                          'leverage':[1,1,1]
#                         })
# account.to_csv('account.csv', index = False)

In [27]:
def close_order(date, signal, trade_nums=[]):
    '''
    根据交易信号，进行盘中交易，并更新记录
    :param signal: 由before_trade()函数得到的交易信号字典
    :param date: 处理时间
    :return:
    '''

    global trade_num
    global init_cash
    global max_cash
    global opt_info
    global underlying_symbol
    per_fee = param_standard(underlying_symbol)['per_fee']
    underlying_prc_his = param_standard(underlying_symbol)['underlying_prc_his']
    underlying_prc_info = underlying_prc_his[underlying_prc_his.index == pd.to_datetime(date).strftime('%Y-%m-%d')]
    opt_quotation_info = pd.read_feather(
        f"{hq.data_path}/quotation/opt/1d/{pd.to_datetime(date).strftime('%Y-%m-%d')}.fea")

    if os.path.exists('position.csv') == False:  # 尚未开始回测, 开仓并创建、更新三个记录表
        if len(signal['trade_opt_code']) > 0:
            print('尚未开仓')
        else:
            print('请输入开仓交易信息')
    else:
        if signal['is_close'] == True:  # 需要平仓
            close_opt_codes = signal['close_opt_code']
            for i, code in enumerate(close_opt_codes):
                position = pd.read_csv('position.csv')
                account = pd.read_csv('account.csv')
                position['datetime'] = position['datetime'].map(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d %H:%M:%S'))
                account['date'] = account['date'].map(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))
                lastest_datetime = position.sort_values(['datetime'], ascending=True).iloc[-1, :]['datetime']
                lastest_date = pd.to_datetime(lastest_datetime).strftime('%Y-%m-%d')
                lastest_date_account = account.sort_values(['date'], ascending=True).iloc[-1, :]['date']
                latest_position = position[position['datetime'] == lastest_datetime]  # 取最新的更新情况
                latest_account = account[account['date'] == lastest_date_account]
                temp_contract_info = opt_info.get_contract_info(code)
                opt_position = latest_position[latest_position['code'] == code]
                close_option = opt_quotation_info[opt_quotation_info['code'] == code]
                trade_price = close_option['settle_price'].tolist()[0]
                trade_num = opt_position['res_num'].tolist()[0] if len(trade_nums)==0 else trade_nums[i]  # 权力方为正，义务方为负
                trade_money = trade_price * trade_num * temp_contract_info['contract_unit'].iloc[0]
                                  + per_fee  * trade_num * (trade_num<0-trade_num>0)
                exercise_price = opt_position['exercise_price'].tolist()[0]
                expire_date = opt_position['expire_date'].tolist()[0]
                upset_expire_days = get_trade_date_gap(underlying_symbol, date, expire_date)
                upset_res_num = opt_position['res_num'].tolist()[0] - trade_num
                # 更新 orderbook
                orderbook_df = pd.DataFrame({'datetime': date, 'code': code, 'underlying_symbol': underlying_symbol,
                                             'exercise_price': exercise_price, 'expire_date': expire_date,
                                             'expire_days': upset_expire_days, 'trade_price': trade_price,
                                             'trade_num': trade_num, 'trade_money': trade_money,
                                             'res_num': upset_res_num, 'init_margin': 0}, index=[0])
                orderbook_df.to_csv('orderbook.csv', mode='a', index=False, header=False)
                # 更新 position
                latest_position['trade_pl'] = 0
                if upset_res_num == 0:  # 剩余持仓为0，直接删除该合约信息
                    drop_index = latest_position[latest_position['code'] == code].index
                    latest_position.drop(latest_position.loc[latest_position['code'] == code].index, inplace=True)
                else:
                    latest_position[latest_position['code'] == code]['res_num'] = upset_res_num
                    if pd.to_datetime(date).strftime('%Y-%m-%d') == lastest_date:  # 日内更新，原值上累加
                        latest_position[latest_position['code'] == code]['trade_pl'] += trade_money 
                    else:
                        latest_position[latest_position['code'] == code]['trade_pl'] = trade_money
                latest_position['datetime'] = pd.to_datetime(date).strftime('%Y-%m-%d %H:%M:%S')
                latest_position['expire_days'] = latest_position.apply(
                    lambda x: get_trade_date_gap(underlying_symbol, pd.to_datetime(date).strftime('%Y-%m-%d'),
                                                 x['expire_date']), axis=1)
                settle_result = pd.merge(latest_position, opt_quotation_info, on='code', how='left')
                latest_position['opt_settle'] = settle_result['settle_price'].values
                latest_position['underlying_close'] = underlying_prc_info['close'].tolist()[0]
                latest_position['maintainance_margin'] = latest_position.apply(
                    lambda x: calculate_maintainance_margin(x['code'], date) * abs(min(x['res_num'], 0)), axis=1)
                latest_position['theo_price'] = latest_position.apply(
                    lambda x: calculate_greeks(x['code'], pd.to_datetime(date).strftime('%Y-%m-%d'))['theo_price'], axis=1)
                latest_position['delta'] = latest_position.apply(
                    lambda x: calculate_greeks(x['code'], pd.to_datetime(date).strftime('%Y-%m-%d'))['delta'], axis=1)
                latest_position['gamma'] = latest_position.apply(
                    lambda x: calculate_greeks(x['code'], pd.to_datetime(date).strftime('%Y-%m-%d'))['gamma'], axis=1)
                latest_position['vega'] = latest_position.apply(
                    lambda x: calculate_greeks(x['code'], pd.to_datetime(date).strftime('%Y-%m-%d'))['vega'], axis=1)
                latest_position['theta'] = latest_position.apply(
                    lambda x: calculate_greeks(x['code'], pd.to_datetime(date).strftime('%Y-%m-%d'))['theta'], axis=1)
                latest_position['rho'] = latest_position.apply(
                    lambda x: calculate_greeks(x['code'], pd.to_datetime(date).strftime('%Y-%m-%d'))['rho'], axis=1)
                latest_position['leverage'] = latest_position.apply(
                    lambda x: calculate_greeks(x['code'], pd.to_datetime(date).strftime('%Y-%m-%d'))['leverage'], axis=1)
                latest_position['hold_pl'] = 0
                position_df = latest_position
                if pd.to_datetime(date).strftime('%Y-%m-%d') == lastest_date:  # 日内更新，覆盖原值
                    position = pd.concat([position[position['datetime'] != lastest_datetime], position_df], ignore_index=True)
                    position.to_csv('position.csv', index=False)
                else:  # 非日内更新, 追加即可
                    position_df['init_margin'] = 0
                    position_df.to_csv('position.csv', index=False, mode='a', header=False)
                # 更新 account
                latest_account['date'] = pd.to_datetime(date).strftime('%Y-%m-%d')
                add_cash = abs(min(latest_account['cash'].tolist()[0] + trade_money, 0)) # 只有义务方买入平仓时才有可能资金不够
                latest_account['add_cash'] = add_cash
                latest_account['total_cash'] = latest_account['add_cash'] + latest_account['init_cash']
                latest_account['hold_pl'] = 0
                latest_account['margin_occup'] = position_df['maintainance_margin'].sum()
                latest_account['margin_account'] += add_cash + trade_money
                latest_account['cash'] = latest_account['margin_account'] - latest_account['margin_occup']
                latest_account['bail_risk_rate'] = latest_account['margin_occup']/latest_account['margin_account']
                latest_account['delta'] = position_df['delta'].sum()
                latest_account['gamma'] = position_df['gamma'].sum()
                latest_account['vega'] = position_df['vega'].sum()
                latest_account['theta'] = position_df['theta'].sum()
                latest_account['rho'] = position_df['rho'].sum()
                latest_account['leverage'] = (position_df['leverage'] * position_df['opt_settle']).sum() / position_df['opt_settle'].sum()
                if pd.to_datetime(date).strftime('%Y-%m-%d') == lastest_date_account:  # 日内更新，覆盖原值
                    account = pd.concat([account[account['date'] != lastest_date_account], latest_account], ignore_index=True)
                    account.to_csv('account.csv', index=False)
                else:  # 非日内更新, 追加即可
                    account.to_csv('account.csv', index=False, mode='a', header=False)
                
# close_order(datetime.datetime(2021, 7, 2, 12, 2, 3).strftime('%Y-%m-%d %H:%M:%S'), dict1, trade_nums=[])

##### 开仓函数

In [10]:
def open_order(date, signal, params): # 开仓
    if os.path.exists('orderbook.csv') == False:  # 如果没有订单簿,则新设
        orderbook = pd.DataFrame(columns=['datetime', 'code', 'underlying_symbol', 'exercise_price', 'expire_date', 'expire_days', 'trade_price', 'trade_size', 'trade_money', 'res_num', 'init_margin'])
        #print(orderbook)
        position = pd.DataFrame(columns=['datetime', 'code', 'underlying_symbol', 'exercise_price', 'expire_date', 'expire_days', 'res_num', 'underlying_close', 'opt_settle', 'hold_pl', 'acc_hold_pl', 'trade_pl', 'init_margin', 'maintainance_margin', 'theo_price', 'delta', 'gamma', 'vega', 'theta', 'rho', 'leverage'])
        #print(position)
        account = pd.DataFrame(columns=['date', 'init_cash', 'add_cash', 'hold_pl', 'margin_occup','cash', 'margin_account', 'bail_risk_rate', 'delta', 'gamma', 'vega', 'theta', 'rho', 'leverage'])
        account_init_line = { 
                             'init_cash': init_cash, 
                             'add_cash': 0,
                             'hold_pl': 0,
                             'cash': init_cash,
                             'margin_occup': 0,
                             'margin_account': init_cash
                    }
        
        
        account = account.append(account_init_line, ignore_index=True)
        #print(account)
    else:
        orderbook = pd.read_csv('orderbook.csv', index_col=0)
        position = pd.read_csv('position.csv', index_col=0)
        account = pd.read_csv('account.csv', index_col=0)
    
    if signal['is_open'] is True:  # 开仓信号为真
        order = pd.DataFrame(index=range(len(signal['trade_opt_code'])), 
                             columns=['datetime', 'code', 'underlying_symbol', 'exercise_price', 'expire_date', 'expire_days', 'trade_price', 'trade_size', 'trade_money', 'res_num', 'init_margin']
                            )
        #print(order)
        # 估计每一组合总成本、保证金并调整最大交易量（不超过现金限额、保证金限额和市场成交量）
        est_trade_money = 0
        est_init_margin = 0
        available_size_single = []  # 单腿最大成交量
        for i in order.index:
            temp_contract_info = opt_info.get_contract_info(code=signal['trade_opt_code'][i])
            daily_info = opt_info.get_daily_info(code=signal['trade_opt_code'][i], date=date)
                
            est_trade_money += trade_num[i]*(daily_info['close'].iloc[0]*temp_contract_info['contract_unit'].iloc[0] + params['per_fee'] *(trade_num[i]>0-trade_num[i]<0))
            est_init_margin += - min(0, trade_num[i]) * calculate_init_margin(signal['trade_opt_code'][i], date)
            available_size_single.append(daily_info['volume'].iloc[0])  # 当天成交量是未来信息
                
        available_size_portfolio = np.floor(min(min(available_size_single), account['cash'].iloc[-1]/max(est_trade_money,est_init_margin)))

        accu_margin = 0
        accu_trade_pl = 0
        for i in order.index:
            #  根据开仓信号建立当天的订单
            order['datetime'].iloc[i] = pd.to_datetime(date)  #  假设在收盘时以收盘价成交
            order['code'].iloc[i] = signal['trade_opt_code'][i]
            temp_contract_info = opt_info.get_contract_info(code=signal['trade_opt_code'][i])
            daily_info = opt_info.get_daily_info(code=signal['trade_opt_code'][i], date=date)
            order['underlying_symbol'].iloc[i] = underlying_symbol
            order['exercise_price'].iloc[i] = temp_contract_info['exercise_price'].iloc[0]
            order['expire_date'].iloc[i] = temp_contract_info['expire_date'].iloc[0]
            order['expire_days'].iloc[i] = len(hq.get_trade_date(start_date=date, end_date=order['expire_date'].iloc[i], only_list=True))
            order['trade_size'].iloc[i] = trade_num[i]*available_size_portfolio
            order['trade_price'].iloc[i] = daily_info['close'].iloc[0]*temp_contract_info['contract_unit'].iloc[0] + params['per_fee'] *(trade_num[i]>0-trade_num[i]<0)
                
                
            order['trade_money'].iloc[i] = -order['trade_size'].iloc[i]*order['trade_price'].iloc[0]
            if signal['trade_opt_code'][i] in position['code']:
                order['res_num'].iloc[i] = position[position['code'] == signal['trade_opt_code'][i]].index[-1]
            else:
                order['res_num'].iloc[i] = order['trade_size'].iloc[i]
            order['init_margin'].iloc[i] = - min(0, order['trade_size'].iloc[i]) * calculate_init_margin(signal['trade_opt_code'][i], date)
            order.drop(order[order['trade_size'] == 0].index, inplace=True)
            
            #  根据order调整position
            intraday_position = {'datetime': order['datetime'].iloc[i],
                              'code': order['code'].iloc[i],
                              'underlying_symbol':  order['underlying_symbol'].iloc[i],
                              'exercise_price': order['exercise_price'].iloc[i],
                              'expire_date': order['expire_date'].iloc[i],
                              'expire_days': order['expire_days'].iloc[i],
                              'res_num': order['res_num'].iloc[i],
                              'trade_pl': order['trade_money'].iloc[i],
                              'init_margin':  order['init_margin'].iloc[i],
                             }
            position = position.append(intraday_position, ignore_index=True)
            
            #  根据position调整account
            #
            accu_margin += intraday_position['init_margin']
            print(accu_margin)
            #
            accu_trade_pl += intraday_position['trade_pl']
            print(accu_trade_pl)
            
        intraday_moving_account = {'date': date, 
                    'init_cash': init_cash, 
                    'margin_occup': account['margin_occup'].iloc[-1]+accu_margin,
                    'cash': account['cash'].iloc[-1]+intraday_position['trade_pl'],
                    'margin_account': account['cash'].iloc[-1]+ account['add_cash'].iloc[-1]+accu_trade_pl
                    }
            
            
        orderbook = orderbook.append(order, ignore_index=True)
        account = account.append(intraday_moving_account, ignore_index=True)
        print(orderbook)    
        print(position)
        print(account)

##### 盘中运行函数

In [11]:
def handle_data(date):
    global underlying_symbol
    global trade_num
    global init_cash
    signal = before_trade(date)  #获取信号
    params = param_standard(underlying_symbol)
    # print(params)
    print(signal)
    close_order(date, signal)
    open_order(date, signal, params)
    
    

In [12]:
handle_data(date='2021-07-05')  

{'is_open': True, 'is_close': False, 'trade_opt_code': ['10003466.XSHG', '10003475.XSHG', '10003466.XSHG'], 'close_opt_code': []}
495869.39999999997
35685.0
495869.39999999997
-35685.0
991738.7999999999
0.0
              datetime           code underlying_symbol exercise_price  \
0  2021-07-05 00:00:00  10003466.XSHG            510050            3.6   
1  2021-07-05 00:00:00  10003475.XSHG            510050            3.6   
2  2021-07-05 00:00:00  10003466.XSHG            510050            3.6   

  expire_date expire_days trade_price trade_size trade_money res_num  \
0  2021-08-25          38       305.0     -117.0     35685.0  -117.0   
1  2021-08-25          38      2009.6      234.0    -71370.0   234.0   
2  2021-08-25          38       305.0     -117.0     35685.0  -117.0   

  init_margin  
0    495869.4  
1         0.0  
2    495869.4  
    datetime           code underlying_symbol  exercise_price expire_date  \
0 2021-07-05  10003466.XSHG            510050             3.6  202